In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
from tensorflow.keras.layers import Input,Dense,Embedding,Flatten,Concatenate
from tensorflow.keras.optimizers import SGD
from sklearn.utils import shuffle

In [7]:
c_names=['userid','movieid','rating','time']

In [8]:
df=pd.read_csv('u.data',names=c_names,sep='\t')

In [9]:
df.head()

,userid,movieid,rating,time
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [13]:
df.userid=pd.Categorical(df.userid)
df['new_user_id'] = df.userid.cat.codes

In [15]:
df.movieid=pd.Categorical(df.movieid)
df['new_movie_id'] = df.movieid.cat.codes

In [16]:
df.head()

,userid,movieid,rating,time,new_user_id,new_movie_id
0,196,242,3,881250949,195,241
1,186,302,3,891717742,185,301
2,22,377,1,878887116,21,376
3,244,51,2,880606923,243,50
4,166,346,1,886397596,165,345


In [18]:
user_id=df['new_user_id'].values
movie_id=df['new_movie_id'].values
rating=df['rating'].values

In [21]:
N=len(set(user_id))
M=len(set(movie_id))
k=10

In [20]:
#model

In [24]:
u=Input(shape=(1,))
m=Input(shape=(1,))
uemb=Embedding(N,k)(u)# op is num_samples,1,k
memb=Embedding(M,k)(m)# ^ same 

uemb = Flatten()(uemb) # now it's (num_samples, K)
memb = Flatten()(memb) # now it's (num_samples, K)

x=Concatenate()([uemb,memb])

x = Dense(400, activation='relu')(x)
x = Dense(400, activation='relu')(x)
x = Dense(400, activation='relu')(x)
x=Dense(1)(x)

In [25]:
from tensorflow.keras.models import Model

In [26]:
model=Model(inputs=[u,m],outputs=x)

In [27]:
model.compile(loss='mse',optimizer=SGD(lr=0.08,momentum=0.9))

In [32]:
user_id, movie_id, rating = shuffle(user_id, movie_id, rating)
Ntrain = int(0.8 * len(rating))
train_user = user_id[:Ntrain]
train_movie = movie_id[:Ntrain]
train_rating = rating[:Ntrain]

test_user = user_id[Ntrain:]
test_movie = movie_id[Ntrain:]
test_rating = rating[Ntrain:]

# center the ratings
avg_rating = train_rating.mean()
train_rating = train_rating - avg_rating
test_rating = test_rating - avg_rating

In [33]:
r = model.fit(
  x=[train_user, train_movie],
  y=train_rating,
  epochs=25,
  batch_size=1024,
  verbose=2, # goes a little faster when you don't print the progress bar
  validation_data=([test_user, test_movie], test_rating),
)

Epoch 1/25
79/79 - 3s - loss: 1.2283 - val_loss: 1.0044
Epoch 2/25
79/79 - 2s - loss: 0.9739 - val_loss: 0.9012
Epoch 3/25
79/79 - 2s - loss: 0.8925 - val_loss: 0.8902
Epoch 4/25
79/79 - 2s - loss: 0.8826 - val_loss: 0.8802
Epoch 5/25
79/79 - 2s - loss: 0.8636 - val_loss: 0.8754
Epoch 6/25
79/79 - 2s - loss: 0.8525 - val_loss: 0.8609
Epoch 7/25
79/79 - 2s - loss: 0.8466 - val_loss: 0.8576
Epoch 8/25
79/79 - 2s - loss: 0.8369 - val_loss: 0.8559
Epoch 9/25
79/79 - 2s - loss: 0.8294 - val_loss: 0.8516
Epoch 10/25
79/79 - 2s - loss: 0.8321 - val_loss: 0.8745
Epoch 11/25
79/79 - 2s - loss: 0.8243 - val_loss: 0.8582
Epoch 12/25
79/79 - 2s - loss: 0.8197 - val_loss: 0.8477
Epoch 13/25
79/79 - 2s - loss: 0.8140 - val_loss: 0.8636
Epoch 14/25
79/79 - 2s - loss: 0.8127 - val_loss: 0.8638
Epoch 15/25
79/79 - 2s - loss: 0.8069 - val_loss: 0.8582
Epoch 16/25
79/79 - 2s - loss: 0.8047 - val_loss: 0.8498
Epoch 17/25
79/79 - 2s - loss: 0.7997 - val_loss: 0.8529
Epoch 18/25
79/79 - 2s - loss: 0.7986 - 